# Données Exif

La plupart des appareils photos récents et téléphones portables enregistrent les photographies avec des données géographiques (longitude, latitude, mais aussi altitude). Si ces données sont lisibles avec la majorité des logiciels photos et d’explorateurs de fichiers, il est également possible d’y accéder avec Python.

Voici l'image que nous allons étudier :
![](images/thorsmork_small.jpg)

La cellule suivante est très importante, elle va permettre d'installer le module pillow qui permet de traiter les images. L'installation peut prendre un peu de temps.

In [ ]:
!pip install pillow

On importe alors les modules nécessaires :

In [ ]:
from PIL import Image # On importe le module Image de la bibliothèque PIL
from PIL.ExifTags import TAGS, GPSTAGS # on importe les modules de gestion des tags EXIF

In [ ]:
img=Image.open("images/thorsmork.jpg") #on charge en mémoire l'image à étudier
img # affiche l'image taille réelle

A l'aide des cellules suivantes, répondre aux questions suivantes sur la fiche réponse :
***

1) Quel appareil photo a pris cette photographie ?

2) la largeur de l'image en pixel

3) la hauteur de l'image en pixel

4) la date et l'heure de la prise de vue

***



On peut connaitre l'ensemble des clés EXIF :
https://www.exiv2.org/tags.html


In [ ]:
exif_data = img._getexif() # permet de récupérer les données EXIF

In [ ]:
exif_data # affiche les données EXIF récupérées

La clé "34853" n'est pas tout le temps présente puisqu'il s'agit des coordonnées (latitude, longitude) de la prise de vue, il faut donc que l'appareil photo intègre un GPS (ce qui est le cas des smartphones), si étudions cette clé "34853", nous obtenons les informations suivantes :

ligne 1 : précise que nous sommes dans l'hémisphère Nord
ligne 2 : nous avons la latitude ((47, 1), (37, 1), (29107360, 1000000)) nous avons ici une latitude en degrés, minute, seconde (ici : 47 degrés 37 minutes et 29,107360 secondes, aussi noté 47°37'29,107360")
ligne 3 : précise que nous sommes à l'ouest (W) du méridien de Greenwich
ligne 4 : nous avons la longitude ((3, 1), (25, 1), (42976570, 1000000) ici aussi la longitude est donnée en degrés, minute, seconde (ici : 3°25'42,976570")

In [ ]:
exif_data[34853] # affiche uniquement les données de la clé 34853 correspondant aux données GPS

In [ ]:
exif_data[34853][1] # affiche l'information de la clé 1

In [ ]:
exif_data[34853][2] # affiche l'information de la clé 2

In [ ]:
exif_data[34853][2][0] # affiche la première information de la clé 2

cette clé correspond à 63 * 1 degré

Pour convertir des (degrés, minute, seconde aussi noté DMS) en "degrés décimaux" (DD) (unité que nous avons utilisée dans le module "cartographie"), il faut appliquer la formule suivante :
DD = degrés+(minute/60)+(seconde/3600)

ATTENTION : pour que la conversion soit correcte, il faut ajouter un signe moins devant la longitude au format DD si nous sommes situés à l'ouest du méridien de Greenwich.

On définit une fonction pour effectuer pour nous la conversion en degrés :
((63, 1), (409847, 10000), (0, 1)) correspond à :

63/1 degrés + (409847 /10000 ) minutes + (0/1) secondes  
soit un calcul à faire :


$$ \frac{63}{1} + \frac{409847}{10000} *\frac{1}{60} + \frac{0}{1} * \frac{1}{3600}$$

In [ ]:
def _en_degres(value): # value est ici un objet particulier : un tuple de tuples ?
    
    d0 = float(value[0][0]) # prend la valeur des degrés en nombre
    d1 = float(value[0][1]) # prend le coefficient multiplicateur
    d = d0 / d1 # on obtient les degrés

    m0 = float(value[1][0]) # prend la valeur des minutes en nombre
    m1 = float(value[1][1])
    m = m0 / m1 # on obtient les minutes

    s0 = float(value[2][0]) # prend la valeur des secondes en nombre
    s1 = float(value[2][1])
    s = s0 / s1 # on récupère les secondes

    return ( d + (m / 60.0) + (s / 3600.0) )

on essaie pour tester si la fonction "fonctionne" :

In [ ]:
_en_degres(exif_data[34853][2])

autres méthodes / essais :

In [ ]:
ret = {}
for tag, value in exif_data.items():
        decoded = TAGS.get(tag, tag)
        ret[decoded] = value
ret

In [ ]:
ret['GPSInfo']


on récupère les infos GPS utiles

In [ ]:
gps_latitude = exif_data[34853][2]
gps_latitude_ref = exif_data[34853][1]
gps_longitude = exif_data[34853][4]
gps_longitude_ref = exif_data[34853][3]

gps_latitude_ref

On convertit les angles en degrés :

In [ ]:
latitude = _en_degres(gps_latitude)
longitude = _en_degres(gps_longitude)



on fait attention au point cardinal de référence :


In [ ]:
if gps_latitude_ref != "N":                     
                latitude = 0 - latitude
if gps_longitude_ref != "E":
                longitude = 0 - longitude


In [ ]:
print(f"la latitude est :{latitude} , la longitude est {longitude} ")

Essai de géolocalisation :

On installe la bibliothèque folium pour utiliser des cartes :

In [ ]:
!pip install folium

In [ ]:
import folium

macarte = folium.Map(location=[latitude,longitude],zoom_start=4) # pour créer la carte
folium.Marker([latitude,longitude]).add_to(macarte) # pour placer un marqueur
macarte # pour afficher la carte que l'on vient de créer

ça marche !

Compléments et essais :

In [ ]:
type(exif_data[34853])

connaitre l'altitude :
ce sont les clés 5 et 6 du dictionnaire exif_data contenant le dictionnaire exif_data[34853]

la clé 5 : GPSAltitudeRef  indique si l'altitude est au dessus ou en dessous du niveau de la mer 

b'\x00'


la clé 6 : GPSAltitude donne l'altitude sous la forme d'un nombre rationnel :  

( 150, 2 ) correspond ainsi à 150/2 = 75 m

GPSAltitudeRef Indicates the altitude used as the reference altitude. If the reference is sea level and the altitude is above sea level, 0 is given. If the altitude is below sea level, a value of 1 is given and the altitude is indicated as an absolute value in the GPSAltitude tag. The reference unit is meters. Note that this tag is BYTE type, unlike other reference tags.

GPSAltitude Indicates the altitude based on the reference in GPSAltitudeRef. Altitude is expressed as one RATIONAL value. The reference unit is meters.

On peut considérer les dictionnaires comme des ensembles de paires clé: valeur, les clés devant être uniques (au sein d'un dictionnaire). 
Une paire d'accolades crée un dictionnaire vide : {}. Placer une liste de paires clé:valeur séparées par des virgules à l'intérieur des accolades ajoute les valeurs correspondantes au dictionnaire ; c'est également de cette façon que les dictionnaires sont affichés.

In [ ]:
hauteur = exif_data[34853][6]
hauteur

In [ ]:
hauteur_m = hauteur[0]/hauteur[1]
print (f"la hauteur en mètres vaut : {round(hauteur_m,1)} m" )

In [ ]:
ref = exif_data[34853][5]
ref

\x00 signifie null , c'est un objet bytes (=octet ?)


In [ ]:
type (ref)

In [ ]:
ref[0] # pour accéder à la valeur du bytes en integer


In [ ]:
int.from_bytes(ref, byteorder='big') # pour convertir un byte en integer

In [ ]:
bytes([0]) # pour convertir un integer en byte , ne pas oublier les crochets